In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import csv
import math
import pickle
import random
from collections import defaultdict

# **Read Inputs**

In [11]:
# Read user data
lastWord = ""
words = defaultdict(dict)
with open('Data/Example/example_data.txt', 'r') as input:
    for line in input:
        line.strip
        if line.startswith("==="):
            words[str(line)[3:len(str(line))-4]] = []
            lastWord = str(line)[3:len(str(line))-4]
        else:
            touchpoints = str(line)[0:len(str(line))-2].split(" ")
            touchpoints[0] = str(line[0:1])
            touchpoints[1] = float(touchpoints[1])
            touchpoints[2] = float(touchpoints[2])
            words[lastWord].append(touchpoints)
words

defaultdict(dict,
            {'Word 0': ['a', 4.819348, 13.72294],
             'Word 1': ['m', 28.266672, 16.68519],
             'Word 2': ['h', 22.114313, 13.22164],
             'Word 3': ['e', 9.809595, 9.48465],
             'Word 4': ['e', 9.809595, 8.57319],
             'Word 5': ['e', 9.296898, 9.16564],
             'Word 6': ['e', 9.296898, 8.70991],
             'Word 7': ['r', 12.407257, 8.80106],
             'Word 8': ['e', 10.014673, 8.84663],
             'Word 9': ['e', 9.8779545, 9.25679],
             'Word 10': ['n', 24.609436, 17.186],
             'Word 11': ['e', 9.125999, 9.39351],
             'Word 12': ['k', 28.027412, 11.07971],
             'Word 13': ['f', 14.389684, 13.72294],
             'Word 14': ['e', 8.750022, 9.0745],
             'Word 15': ['y', 19.790089, 8.3909],
             'Word 16': ['e', 9.433618, 8.98335],
             'Word 17': ['e', 9.467797, 7.7073],
             'Word 18': ['o', 30.761795, 7.798455],
             'Word 19': ['t', 

In [ ]:
# Read dictionary
with open('Data/unigram.dict', 'rb') as unigramModelFile:
    unigramModel = pickle.load(unigramModelFile)
unigramModelFile.close()

# Read keyboard data
keyboard_raw = pd.read_csv("Data/keyboard.csv")
keyboard = keyboard_raw[['key', 'x_mm', 'y_mm']]
keyboard

# **Unigram Language Model Decoder**

In [ ]:
# Keyboard size and dual Gaussian model parameters
key_width = 3
key_height = 4
a = 2.403
b = 0.017
c = 2.295
d = 0.016

def get_likelihood(p, mu, sigma):
    """
    TODO: Calculate the likelihood that a touch point p is from the 2D Gaussian distribution N(mu, sigma)
    """
    lik = 
    return lik

def is_letter(p, letter):
    """
    TODO: Determine if touch point p is located inside the boundary of the key: letter
    """
    return False 

def get_literal_string(touchpoints):
    """ 
    TODO: Compute the literal string using is_letter(p, letter) method for a collection of touch points that represents a word. 
          If a touch point does not fall inside any key boundary, use '?' to represent the corresponding character.
    """
    literal_string = ""
    
    return literal_string

In [ ]:
def unigram_lm_decoder(touchpoints):
    """
    A language decoder that uses the dual Gaussian touch point spatial disrtibution model and a unigram language model.
    Input: a list/collection of touch points that represents a certain word
    Output: the decoded word for the input
    
    TODO: Step a --- Get all possible words and their corresponding probabilities from the dictionary. 
          Use the length of the correct word to filter possible words
          You may also use the first and/or the last touchpoint to further narrow down possible words
    """
    
    possible_words = []
    
    # Calculate p(w|s_1, s_2, ... s_n) ~ p(s_1, s_2, ..., s_n|w)*p(w) = \Pi(p(s_i|c_i))p(w) for each possible word
    p_w_s = []                  # Holds p(w|s_1, s_2, ..., s_n) for all possible words
    for item in possible_words:
        word =                    # TODO: The current possible word
        p_w =                     # TODO: Probability of the current possible word in the unigram language model
        p_s_w = 1                 # Holds p(s_1, s_2, ..., s_n|w) for the current possible word

        for j, letter in enumerate(list(word)):
            # TODO: Step b --- Apply the spatial model to get p(s_i|c_i)
            mu =
            sigma =
            p_s_c = 
            
            # TODO: Step b --- Multiply the current p(s_i|c_i) to p(s_1, s_2, ..., s_n|W)
        
        # TODO: Step c --- Calculate p(w|s_1, s_2, ... s_n) from p(s_1, s_2, ..., s_n|w) and p(w). Append the result to list
    
    # TODO: Step d ---- Choose word by the maximum of p(w|s_1, s_2, ..., s_n)
    decoded_word = ""
    
    return decoded_word

In [ ]:
decoded_success_count = 0
literal_success_count = 0
decoded_words = []
literal_strings = []
correct_words = []
for touchpoints in words:
    """
    TODO: Use above methods to compute the correct word, decoded word, and the literal string for each touch point collection
          Append results to the corresponding list
          Update the decoded words/literal strings success count. 
              --- If the decoded word/literal string is the same as correct word, increase 1 to decoded words/literal strings success count
    """
    
# TODO: calculate the success rate for both the decoded words and the literal strings using the docoded word/literal string success count

# TODO: Write to results.txt
with open("results.txt", 'w') as output:
    # The first line: success_rate(decoded_words), success_rate(literal_strings)
    for i in range(len(correct_words)):
        # Each line after: correct_word, decoded_word, literal_string

output.close()